<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rdkit deepsmiles

In [8]:
!pip3 install torchinfo

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses huggingface_hub

In [ ]:
!pip install transformers -U

In [ ]:
!wget http://file.weasoft.com/80k.csv

In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
import deepsmiles
import numpy as np
import pylab
converter = deepsmiles.Converter(rings=True, branches=True)
def deepsmiles_to_img(ds):
   img = np.array(Draw.MolToImage(Chem.MolFromSmiles(converter.decode(ds)), size=(400,400)).convert("L", dither=None).convert("RGB"))
   img = np.where(img<253, 0, 1) * img
   return img

def smiles_to_img(smiles):
  return deepsmiles_to_img(converter.encode(smiles))

In [2]:
import pandas
csv = pandas.read_csv("80k.csv")

In [3]:
# prompt: convert all the data in canonicalsmiles column using converter.encode function and store them in a new array

smiles_arr = []
for smiles in csv['canonicalsmiles']:
  smiles_arr.append(converter.encode(smiles))


In [4]:
import torch
import torchvision

In [47]:
image_encoder = torchvision.models.efficientnet_v2_l(weights="DEFAULT")

In [10]:
smiles_encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-large-uncased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [11]:
chars = list(set("".join(smiles_arr)))

In [12]:
tokens = dict.fromkeys(chars)
for i, char in enumerate(chars):
  tokens[char] = i

In [13]:
reversed_mapping = {}
for i, char in enumerate(chars):
  reversed_mapping[i] = char

In [14]:
for i, smiles in enumerate(smiles_arr):
  smiles_arr[i] = [len(chars)]+[tokens[char] for char in smiles]

In [ ]:
image_encoder = image_encoder.features

In [54]:
image_encoder.classifier = torch.nn.Linear(1280, 1024)

In [ ]:
import torchinfo
torchinfo.summary(image_encoder, (1, 3, 224, 224))

In [53]:
import torchinfo
smiles_encoder(torch.tensor([[1,2,3]])).last_hidden_state[:,0,:].flatten()

torch.Size([1, 1024])

In [80]:
CEL = torch.nn.CrossEntropyLoss()
class ComtrastLearning(torch.nn.Module):
  def __init__(self, image_encoder, smiles_encoder):
    super().__init__()
    self.image_encoder = image_encoder
    self.smiles_encoder = smiles_encoder
    self.smiles_proj = torch.nn.Linear(1024, 1024)
    # https://discuss.pytorch.org/t/how-could-i-create-a-module-with-learnable-parameters/28115
    self.t = torch.nn.Parameter(torch.randn(1))
    self.t.requires_grad = True
  def forward(self, image, smiles):
    image_embedding = self.image_encoder(image)
    image_embedding = torch.flatten(image_embedding, start_dim=2, end_dim=3)
    smiles_embedding = self.smiles_encoder(smiles).last_hidden_state[:,0,:]
    smiles_embedding = self.smiles_proj(smiles_embedding)
    n = smiles_embedding.shape()[0]
    a = torch.broadcast_to(image_embedding, (n, n, -1))
    b = torch.broadcast_to(smiles_embedding, (n, n, -1)).permute(1,0,2)
    logits = torch.nn.functional.cosine_similarity(a, b, dim=-1) * torch.exp(self.t)
    return logits

In [79]:
n = 2
a = torch.tensor([[1.,2.,3.],[2,5,6]])
b = torch.tensor([[2,5,6],[1.,2.,3.]])
a = torch.broadcast_to(a, (n, n, -1))
b = torch.broadcast_to(b, (n, n, -1)).permute(1,0,2)
torch.nn.functional.cosine_similarity(a, b, dim=-1)


tensor([[0.9945, 1.0000],
        [1.0000, 0.9945]])